In [ ]:
#若没有安装d2l需要取消下行注释
#!pip install d2l
import torch
import torchvision
from torchvision import transforms
from torch import nn
from d2l import torch as d2l
from torch.utils import data
import matplotlib.pyplot as plt
from torch.nn import functional as F
import numpy as np
#图片增广
normalize = torchvision.transforms.Normalize(
    [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
train_imgs=transforms.Compose([torchvision.transforms.RandomResizedCrop(224),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    normalize])
test_imgs=transforms.Compose([torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    normalize])
#读取图片数据集
def read_imgs(augs,is_train,batch_size):
    dataset=torchvision.datasets.CIFAR10(root="./CIFAR10_img",transform=augs,
                                        download=False,train=is_train)
    return data.DataLoader(dataset,shuffle=is_train,batch_size=batch_size,num_workers=2)
#图片标签的存放
def get_CIFAR10_labels(labels):
    text_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                   'dog', 'frog', 'house', 'ship', 'truck']
    return [text_labels[int(i)] for i in labels]

In [ ]:
##### 定义ResNet-X块
# class Residual(nn.Module):
#     def __init__(self,num_channels,use_1x1conv=False,strides=1):
#         super().__init__()
#         self.Conv1=nn.LazyConv2d(num_channels,kernel_size=3,stride=strides,padding=1)
#         self.Conv2=nn.LazyConv2d(num_channels,kernel_size=3,padding=1)
#         if use_1x1conv:
#             self.Conv3=nn.LazyConv2d(num_channels,kernel_size=1,stride=strides)
#         else:
#             self.Conv3=None
#         self.bn1=nn.LazyBatchNorm2d(num_channels)
#         self.bn2=nn.LazyBatchNorm2d(num_channels)
#     def forward(self,X):
#         Y=F.relu(self.bn1(self.Conv1(X)))
#         Y=self.bn2(self.Conv2(Y))
#         if self.Conv3:
#             X=self.Conv3(X)
#         Y+=X
#         return F.relu(Y)
# #定义ResNet-X模型
# def ResNet18(num_classes,in_channels=1):
#     def ResNet_model(in_channels,out_channels,block_num,first_block=False):
#         ren=[]
#         for i in range(block_num):
#             if i==0 and not first_block:
#                 ren.append(Residual(out_channels,use_1x1conv=True,strides=2))
#             else:
#                 ren.append(Residual(out_channels))
#         return ren
#     b1=nn.Sequential(nn.Conv2d(3,64,kernel_size=7,stride=2,padding=3),
#                     nn.BatchNorm2d(64,eps=1e-05, momentum=0.1),nn.ReLU(),
#                     nn.MaxPool2d(kernel_size=3,stride=2,padding=1))  
#     b2=nn.Sequential(*ResNet_model(64,64,2,first_block=True))
#     b3=nn.Sequential(*ResNet_model(64,128,2))
#     b4=nn.Sequential(*ResNet_model(128,256,2))
#     b5=nn.Sequential(*ResNet_model(256,512,2))
#     net=nn.Sequential(b1,b2,b3,b4,b5,nn.AdaptiveAvgPool2d((1,1)),
#                      nn.Flatten(),nn.Linear(512,num_classes))
#     return net
#定义一个存放数据的lei
class Addter():
    def __init__(self,n):
        self.data=[0.0]*n
    def add(self,*args):
        self.data=[a + float(b) for a,b in zip(self.data,args)]    
# 多GPU训练
a=[]
train_loss=[]
train_ac=[]
test_ac=[]
#图片展示
def show_images(imgs, num_rows, num_cols, titles=None, scale=1.5):
    figsize = (num_cols * scale, num_rows * scale)
    _,axes =plt.subplots(num_rows, num_cols, figsize=figsize)
    axes = axes.flatten()
    for i, (ax, img) in enumerate(zip(axes, imgs)):
        if torch.is_tensor(img):
            np_img = img.cpu().numpy()  # tensor --> numpy
            ax.imshow(np.transpose(np_img,(1,2,0)))#显示位置转换如（size,size,channels）=>(channels,size,size)
        else:
            ax.imshow(img)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        if titles:
            ax.set_title(titles[i])
    return axes
#前向后向
def train_gpu(net,X,y,loss,updater,devices):
        X,y=X.to(devices[0]),y.to(devices[0])
        net.train()
        updater.zero_grad()
        y_hat=net(X)
        l=loss(y_hat,y)
        l.sum().backward()
        updater.step()
        train_loss_sum=l.sum()
        train_acc_sum=d2l.accuracy(y_hat,y)
        return train_loss_sum,train_acc_sum
    #训练
def train(net,train_iter,test_iter,epochs,lr,loss,updater,devices): 
    timer=d2l.Timer()
    scheduler=torch.optim.lr_scheduler.StepLR(updater,lr_epoch,lr_decay)
    net=nn.DataParallel(net,device_ids=devices).to(devices[0])
    for epoch in range(epochs):
        addter=Addter(4)
        for i,(X,y) in enumerate(train_iter):
            timer.start()
            l,acc=train_gpu(net,X,y,loss,updater,devices)
            timer.stop()
            addter.add(l,acc,y.shape[0],y.numel())
            train_l=addter.data[0]/addter.data[2]
            train_acc=addter.data[1]/addter.data[3]
        test_acc=d2l.evaluate_accuracy_gpu(net, test_iter)
        scheduler.step()
        a.append(epoch+1)
        train_loss.append(train_l)
        train_ac.append(train_acc)
        test_ac.append(test_acc)
        print(f"epoch:{epoch+1},train_loss:{train_l:.3f},tain_acc:{train_acc:.3f},test_acc:{test_acc:.3f},"
              f"time:{timer.sum()/60:.2f}m")
 #预测图片 
def predict(net,test_iter,devices,n=10):
    for X,y in test_iter:
        break
    X,y=X.to(devices[0]),y.to(devices[0])
    trues=get_CIFAR10_labels(y)
    preds=get_CIFAR10_labels(torch.argmax(net(X),axis=1))
    titles=[true+"\n"+"out:"+pred for true, pred in zip(trues,preds)]
    show_images(torch.reshape(X[0:n],(n,3,224,224)),1,10,titles=titles[0:n])  
#迁移学习（微调）
def move_learning(net,lr,epochs,batch_size,lr_epoch,lr_decay,w_decay,param_group=True):
    train_iter=read_imgs(train_imgs,True,batch_size)
    test_iter=read_imgs(test_imgs,False,batch_size)
    loss=nn.CrossEntropyLoss(reduction="none")
    if param_group:
        params = [param for name, param in net.named_parameters()
             if name not in ["fc.weight", "fc.bias"]]
        updater = torch.optim.SGD([{'params': params},
                                   {'params': net.fc.parameters(),
                                    'lr': lr * 10}],
                                lr=lr, weight_decay=w_decay)
    train(net,train_iter,test_iter,epochs,lr,loss,updater,d2l.try_all_gpus())
    # #可视化
    plt.plot(a,train_loss,label="train_loss")
    plt.plot(a,train_ac,label="train_acc")
    plt.plot(a,test_ac,label="test_acc")
    plt.grid()
    plt.title("ResNet-50")
    plt.xlabel("epochs")
    plt.legend() 
    plt.show()
    #预测
    predict(net,test_iter,d2l.try_all_gpus())            
lr,epochs,batch_size,lr_epoch,lr_decay,w_decay=1e-4,10,32,2,0.9,1e-4
net=torchvision.models.resnet50(pretrained=True)
net.fc=nn.Linear(net.fc.in_features,10)
nn.init.xavier_uniform_(net.fc.weight)
print("training")
move_learning(net,lr,epochs,batch_size,lr_epoch,lr_decay,w_decay,param_group=True)
#保存模型参数
torch.save(net,'resnet50.pth')